### OpenFIBSEM DB

sqlite3 database for storing metadata about FIBSEM datasets.

https://www.sqlitetutorial.net/sqlite-python

In [ ]:
%load_ext autoreload
%autoreload 2


import sqlite3
from fibsem.db.util import *

### Create TABLE


In [ ]:
# create a database connection
conn = create_connection(database)

# create tables
if conn is not None:
    # create projects table
    create_table(conn, sql_create_projects_table)

    # create tasks table
    create_table(conn, sql_create_tasks_table)
else:
    print("Error! cannot create the database connection.")


#### INSERT DATA

In [ ]:
import sqlite3
from sqlite3 import Error




# create a database connection
conn = create_connection(database)
with conn:
    # create a new project
    project = ('Cool App with SQLite & Python', '2015-01-01', '2015-01-30');
    project_id = create_project(conn, project)

    # tasks
    task_1 = ('Analyze the requirements of the app', 1, 1, project_id, '2015-01-01', '2015-01-02')
    task_2 = ('Confirm with user about the top requirements', 1, 1, project_id, '2015-01-03', '2015-01-05')

    # create tasks
    create_task(conn, task_1)
    create_task(conn, task_2)



### Update Task

In [ ]:

# create a database connection
conn = create_connection(database)
with conn:
    update_task(conn, (2, '2015-01-04', '2015-01-06', 2))


### Delete Task


In [ ]:
conn = create_connection(database)
with conn:
    delete_task(conn, 1);
    # delete_all_tasks(conn);

### Query Data


In [ ]:
# create a database connection
conn = create_connection(database)
with conn:
    print("1. Query task by priority:")
    select_task_by_priority(conn, 1)

    print("2. Query all tasks")
    select_all_tasks(conn)

## PANDAS


In [ ]:
import pandas as pd

conn = create_connection(database)
df = pd. read_sql('SELECT * from tasks', conn)

In [ ]:
display(df)

In [ ]:
# write the data to a sqlite table

df = pd.DataFrame({"id": 5, "name": "test", "priority": 3, "status_id": 3, "project_id": 3, "begin_date": '2016-01-12', "end_date": '2016-02-24'}, 
index=[0])

display(df)
df.to_sql('tasks', conn, if_exists='append', index = False)

In [ ]:
df = pd. read_sql('SELECT * from tasks', conn)

In [ ]:
display(df)

DATABASE

--- EXPERIMENT MANAGMENT ---

project
- id
- name
- user_id

user:
- id
- name

experiment
- id
- name
- project_id
- user_id
- sample_id
- path

sample
- id
- name



---- Analytics ----

ml
detection
alignment
interaction
history
steps


In [ ]:

# MILESTONE 1
# TODO: initialise database script
# TODO: migrate ml data collection to store in db
# TODO: setup projects, experiment, user tables
# TODO: separate ETL from statistics / analytics application
# TODO: enable user to select project, experiment
# TODO: enable multi-experiment analytics
# TODO: fix experiment to dataframe function, make actually useful


In [3]:
%load_ext autoreload
%autoreload 2

import datetime
import sqlite3
from fibsem.db.util import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
SQL_CREATE_PROJECTS_TABLE = """ CREATE TABLE IF NOT EXISTS projects (
                                    id INTEGER PRIMARY KEY,
                                    name VARCHAR(100) NOT NULL,
                                    date TIMESTAMP NOT NULL,
                                    user VARCHAR(100) NOT NULL
                                ); """

SQL_CREATE_USERS_TABLES = """CREATE TABLE IF NOT EXISTS users (
                                id INTEGER PRIMARY KEY,
                                name VARCHAR(100) NOT NULL,
                                email VARCHAR(100) NOT NULL,
                                password VARCHAR(100) NOT NULL
                            );"""


SQL_CREATE_SAMPLES_TABLE = """CREATE TABLE IF NOT EXISTS samples (
                                id INTEGER PRIMARY KEY,
                                name VARCHAR(100) NOT NULL,
                                project_id INTEGER NOT NULL,
                                date TIMESTAMP NOT NULL,
                                user VARCHAR(100) NOT NULL,
                                FOREIGN KEY (project_id) REFERENCES projects (id)
                                );"""


SQL_CREATE_EXPERIMENTS_TABLE = """CREATE TABLE IF NOT EXISTS experiments (
                                id INTEGER PRIMARY KEY,
                                name VARCHAR(100) NOT NULL,
                                project_id INTEGER NOT NULL,
                                date TIMESTAMP NOT NULL,
                                user VARCHAR(100) NOT NULL,
                                sample_id INTEGER NOT NULL,
                                program VARCHAR(100) NOT NULL,
                                method VARCHAR(100) NOT NULL,
                                path VARCHAR(100) NOT NULL,
                                FOREIGN KEY (project_id) REFERENCES projects (id)
                                FOREIGN KEY (sample_id) REFERENCES samples (id)
                            );"""


# create / connect to db
database = "fibsem.db"
conn = create_connection(database)

# create tables
create_table(conn, SQL_CREATE_PROJECTS_TABLE)
create_table(conn, SQL_CREATE_USERS_TABLES)
create_table(conn, SQL_CREATE_SAMPLES_TABLE)
create_table(conn, SQL_CREATE_EXPERIMENTS_TABLE)




### Create Project

In [ ]:
import datetime

# create_project(conn, 
#     ("WAFFLE-METHOD-DEVELOPMENT", datetime.datetime.now(), "patrick")
# )

### CREATE USER

In [ ]:
create_user(conn, 
    ('hannah', 'hannah.siems@monash.edu', 'password')
)

### CREATE SAMPLE

In [ ]:
create_sample(
    conn, 
    ("WAFFLE-01", 1, datetime.datetime.now(), "hannah")
)

### CREATE EXPERIMENT



In [ ]:
create_experiment(conn,

    ("WAFFLE-15082023", 1, datetime.datetime.now(), "hannah", 1)

)

In [ ]:
import pandas as pd
print("PROJECTS")
df = pd. read_sql('SELECT * from projects', conn)
display(df)

print("USERS")
df = pd. read_sql('SELECT * from users', conn)
display(df)

print("SAMPLES")
df = pd. read_sql('SELECT * from samples', conn)
display(df)

print("EXPERIMENTS")
df = pd. read_sql('SELECT * from experiments', conn)
display(df)

In [ ]:
### REPLACE ? --> UPSERT?

# might not want to use, replaces experiment?

### Sync Experiments


In [5]:
%load_ext autoreload
%autoreload 2
from autolamella import config as cfg
import os

import pandas as pd
from autolamella.structures import Experiment
from autolamella.tools.data import calculate_statistics_dataframe

EXPERIMENTS = [
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-02-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-15082023",
]

for PATH in EXPERIMENTS:
    
    print(PATH)   

    exp = Experiment.load(os.path.join(PATH, "experiment.yaml"))
    df = exp.to_dataframe_v2()

    # format date as datetime
    df["date"] = pd.to_datetime(df["date"], unit="s")
    df["date"] = pd.to_datetime(df["date"], utc=True)

    # drop experiment_id
    df = df.drop(columns=["experiment_id", "num_lamella"])
    df["project_id"] = 1
    df["sample_id"] = 1
    df["user"] = "hannah"
    df["method"] = "waffle"
    
    display(df)

    df.to_sql('experiments', conn, if_exists='append', index = False)

    print("------------------")





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-09082023


,name,path,date,program,method,project_id,sample_id,user
0,HANNAH-WAFFLE-01-09082023,C:/Users/Admin/Github/autolamella/autolamella/...,2023-08-09 00:07:42.963713792+00:00,AutoLamella,waffle,1,1,hannah


------------------
/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-02-09082023


,name,path,date,program,method,project_id,sample_id,user
0,HANNAH-WAFFLE-02-09082023,C:/Users/Admin/Github/autolamella/autolamella/...,2023-08-09 06:48:25.088992+00:00,AutoLamella,waffle,1,1,hannah


------------------
/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-15082023


,name,path,date,program,method,project_id,sample_id,user
0,HANNAH-WAFFLE-01-15082023,C:/Users/Admin/Github/autolamella/autolamella/...,2023-08-14 23:59:41.935888896+00:00,AutoLamella,waffle,1,1,hannah


------------------


In [6]:
df = pd.read_sql("SELECT * FROM [experiments]", conn)
display(df)

,id,name,project_id,date,user,sample_id,program,method,path
0,1,HANNAH-WAFFLE-01-09082023,1,2023-08-09 00:07:42.963713,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...
1,2,HANNAH-WAFFLE-02-09082023,1,2023-08-09 06:48:25.088992,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...
2,3,HANNAH-WAFFLE-01-15082023,1,2023-08-14 23:59:41.935888,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...


In [21]:

PROJECT_NAME = "WAFFLE-METHOD-DEVELOPMENT"

df = pd.read_sql(f"SELECT * FROM projects WHERE name='{PROJECT_NAME}'", conn)
PROJECT_IDS = df["id"].values

df = pd.read_sql(f"SELECT * FROM experiments WHERE project_id={PROJECT_IDS[0]}", conn)

display(df)

[1]


,id,name,project_id,date,user,sample_id,program,method,path
0,1,HANNAH-WAFFLE-01-09082023,1,2023-08-09 00:07:42.963713,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...
1,2,HANNAH-WAFFLE-02-09082023,1,2023-08-09 06:48:25.088992,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...
2,3,HANNAH-WAFFLE-01-15082023,1,2023-08-14 23:59:41.935888,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...


In [29]:

SQL_CREATE_HISTORY_TABLE = """CREATE TABLE IF NOT EXISTS history (
                                id INTEGER PRIMARY KEY,
                                petname VARCHAR(100) NOT NULL,
                                stage VARCHAR(100) NOT NULL,
                                start TIMESTAMP NOT NULL,
                                end TIMESTAMP NOT NULL,
                                duration FLOAT NOT NULL,
                                experiment_id INTEGER NOT NULL,
                                FOREIGN KEY (experiment_id) REFERENCES experiment (id)
                                
                            );"""


# create / connect to db
database = "fibsem.db"
conn = create_connection(database)

# create tables
create_table(conn, SQL_CREATE_PROJECTS_TABLE)

In [35]:
%load_ext autoreload
%autoreload 2
from autolamella import config as cfg
import os

import pandas as pd
from autolamella.structures import Experiment
from autolamella.tools.data import calculate_statistics_dataframe

EXPERIMENTS = [
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-02-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-15082023",
]


df_exp = pd.read_sql("SELECT * FROM experiments", conn)
display(df_exp)

for PATH in EXPERIMENTS:
    
    print(PATH)   

    dfs = calculate_statistics_dataframe(PATH)
    df = dfs[1]

    # format date as datetime
    df["start"] = pd.to_datetime(df["start"], unit="s")
    df["start"] = pd.to_datetime(df["start"], utc=True)

    df["end"] = pd.to_datetime(df["start"], unit="s")
    df["end"] = pd.to_datetime(df["start"], utc=True)


    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    print(EXP_ID)
    df["experiment_id"] = EXP_ID
    # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])
    
    display(df)


    df.to_sql('history', conn, if_exists='append', index = False)

    print("------------------")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,id,name,project_id,date,user,sample_id,program,method,path
0,1,HANNAH-WAFFLE-01-09082023,1,2023-08-09 00:07:42.963713,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...
1,2,HANNAH-WAFFLE-02-09082023,1,2023-08-09 06:48:25.088992,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...
2,3,HANNAH-WAFFLE-01-15082023,1,2023-08-14 23:59:41.935888,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...


/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-09082023
--------------------------------------------------------------------------------


,lamella,stage,step,timestamp,step_n,duration,exp_name,exp_id
0,01-busy-mite,SetupTrench,STARTED,1.691540e+09,0,0.0,HANNAH-WAFFLE-01-09082023,305ba29c-1608-4cce-84ca-c359fbc3746a
1,02-prime-guppy,SetupTrench,STARTED,1.691540e+09,1,0.0,HANNAH-WAFFLE-01-09082023,305ba29c-1608-4cce-84ca-c359fbc3746a
2,03-witty-sole,SetupTrench,STARTED,1.691540e+09,2,1.0,HANNAH-WAFFLE-01-09082023,305ba29c-1608-4cce-84ca-c359fbc3746a
3,04-superb-bunny,SetupTrench,STARTED,1.691540e+09,3,0.0,HANNAH-WAFFLE-01-09082023,305ba29c-1608-4cce-84ca-c359fbc3746a
4,05-steady-donkey,SetupTrench,STARTED,1.691540e+09,4,0.0,HANNAH-WAFFLE-01-09082023,305ba29c-1608-4cce-84ca-c359fbc3746a
...,...,...,...,...,...,...,...,...
269,06-united-urchin,MillRoughCut,REFERENCE_IMAGES,1.691562e+09,269,11.0,HANNAH-WAFFLE-01-09082023,305ba29c-1608-4cce-84ca-c359fbc3746a
270,06-united-urchin,MillRoughCut,FINISHED,1.691562e+09,270,8.0,HANNAH-WAFFLE-01-09082023,305ba29c-1608-4cce-84ca-c359fbc3746a
271,08-funny-fox,MillRoughCut,STARTED,1.691562e+09,271,0.0,HANNAH-WAFFLE-01-09082023,305ba29c-1608-4cce-84ca-c359fbc3746a
272,08-funny-fox,MillRoughCut,ALIGN_LAMELLA,1.691562e+09,272,6.0,HANNAH-WAFFLE-01-09082023,305ba29c-1608-4cce-84ca-c359fbc3746a


/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-02-09082023
--------------------------------------------------------------------------------


,lamella,stage,step,timestamp,step_n,duration,exp_name,exp_id
0,01-true-shark,SetupTrench,STARTED,1.691564e+09,0,21.0,HANNAH-WAFFLE-02-09082023,0c0000d9-872f-4c52-b680-634cf7be1904
1,01-true-shark,SetupTrench,FINISHED,1.691564e+09,1,0.0,HANNAH-WAFFLE-02-09082023,0c0000d9-872f-4c52-b680-634cf7be1904
2,01-true-shark,ReadyTrench,STARTED,1.691564e+09,2,0.0,HANNAH-WAFFLE-02-09082023,0c0000d9-872f-4c52-b680-634cf7be1904
3,01-true-shark,ReadyTrench,FINISHED,1.691564e+09,3,12.0,HANNAH-WAFFLE-02-09082023,0c0000d9-872f-4c52-b680-634cf7be1904
4,01-true-shark,MillTrench,STARTED,1.691564e+09,4,0.0,HANNAH-WAFFLE-02-09082023,0c0000d9-872f-4c52-b680-634cf7be1904
...,...,...,...,...,...,...,...,...
57,01-true-shark,MillPolishingCut,REFERENCE_IMAGES,1.691568e+09,57,11.0,HANNAH-WAFFLE-02-09082023,0c0000d9-872f-4c52-b680-634cf7be1904
58,01-true-shark,MillPolishingCut,FINISHED,1.691568e+09,58,0.0,HANNAH-WAFFLE-02-09082023,0c0000d9-872f-4c52-b680-634cf7be1904
59,01-true-shark,Finished,STARTED,1.691568e+09,59,0.0,HANNAH-WAFFLE-02-09082023,0c0000d9-872f-4c52-b680-634cf7be1904
60,01-true-shark,Finished,FINISHED,1.691568e+09,60,0.0,HANNAH-WAFFLE-02-09082023,0c0000d9-872f-4c52-b680-634cf7be1904


/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-15082023
--------------------------------------------------------------------------------


,lamella,stage,step,timestamp,step_n,duration,exp_name,exp_id
0,01-fast-lark,SetupTrench,STARTED,1.692058e+09,0,0.0,HANNAH-WAFFLE-01-15082023,1f3f9869-ec6b-4c4b-a978-f1f4cfd52767
1,02-needed-toad,SetupTrench,STARTED,1.692058e+09,1,1.0,HANNAH-WAFFLE-01-15082023,1f3f9869-ec6b-4c4b-a978-f1f4cfd52767
2,03-boss-shrimp,SetupTrench,STARTED,1.692058e+09,2,0.0,HANNAH-WAFFLE-01-15082023,1f3f9869-ec6b-4c4b-a978-f1f4cfd52767
3,04-wired-colt,SetupTrench,STARTED,1.692058e+09,3,0.0,HANNAH-WAFFLE-01-15082023,1f3f9869-ec6b-4c4b-a978-f1f4cfd52767
4,05-grown-turtle,SetupTrench,STARTED,1.692058e+09,4,0.0,HANNAH-WAFFLE-01-15082023,1f3f9869-ec6b-4c4b-a978-f1f4cfd52767
...,...,...,...,...,...,...,...,...
395,08-free-magpie,Finished,STARTED,1.692087e+09,395,1.0,HANNAH-WAFFLE-01-15082023,1f3f9869-ec6b-4c4b-a978-f1f4cfd52767
396,08-free-magpie,Finished,FINISHED,1.692087e+09,396,0.0,HANNAH-WAFFLE-01-15082023,1f3f9869-ec6b-4c4b-a978-f1f4cfd52767
397,09-humble-alien,Finished,STARTED,1.692087e+09,397,0.0,HANNAH-WAFFLE-01-15082023,1f3f9869-ec6b-4c4b-a978-f1f4cfd52767
398,09-humble-alien,Finished,FINISHED,1.692087e+09,398,0.0,HANNAH-WAFFLE-01-15082023,1f3f9869-ec6b-4c4b-a978-f1f4cfd52767


In [37]:


SQL_CREATE_STEPS_TABLE = """CREATE TABLE IF NOT EXISTS steps (
                                id INTEGER PRIMARY KEY,
                                petname VARCHAR(100) NOT NULL,
                                stage VARCHAR(100) NOT NULL,
                                step VARCHAR(100) NOT NULL,
                                step_n INTEGER NOT NULL,
                                timestamp TIMESTAMP NOT NULL,
                                duration FLOAT NOT NULL,
                                experiment_id INTEGER NOT NULL,
                                FOREIGN KEY (experiment_id) REFERENCES experiment (id)
                                
                            );"""

# create / connect to db
database = "fibsem.db"
conn = create_connection(database)

# create tables
create_table(conn, SQL_CREATE_STEPS_TABLE)

In [40]:
%load_ext autoreload
%autoreload 2
from autolamella import config as cfg
import os

import pandas as pd
from autolamella.structures import Experiment
from autolamella.tools.data import calculate_statistics_dataframe

EXPERIMENTS = [
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-02-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-15082023",
]


df_exp = pd.read_sql("SELECT * FROM experiments", conn)
display(df_exp)

for PATH in EXPERIMENTS:

    dfs = calculate_statistics_dataframe(PATH)
    df = dfs[3]

    # display(df)
    # # format date as datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    print(EXP_ID)
    df["experiment_id"] = EXP_ID
    # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])

    # rename lamella to petname
    df = df.rename(columns={"lamella": "petname"})

    # replace NA with 0
    df = df.fillna(0)
    
    display(df.head())

    df.to_sql('steps', conn, if_exists='append', index = False)

    print("------------------")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,id,name,project_id,date,user,sample_id,program,method,path
0,1,HANNAH-WAFFLE-01-09082023,1,2023-08-09 00:07:42.963713,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...
1,2,HANNAH-WAFFLE-02-09082023,1,2023-08-09 06:48:25.088992,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...
2,3,HANNAH-WAFFLE-01-15082023,1,2023-08-14 23:59:41.935888,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...


/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-09082023
--------------------------------------------------------------------------------
1


,petname,stage,step,timestamp,step_n,duration,experiment_id
0,01-busy-mite,SetupTrench,STARTED,2023-08-09 00:15:57+00:00,0,0.0,1
1,02-prime-guppy,SetupTrench,STARTED,2023-08-09 00:15:57+00:00,1,0.0,1
2,03-witty-sole,SetupTrench,STARTED,2023-08-09 00:15:57+00:00,2,1.0,1
3,04-superb-bunny,SetupTrench,STARTED,2023-08-09 00:15:58+00:00,3,0.0,1
4,05-steady-donkey,SetupTrench,STARTED,2023-08-09 00:15:58+00:00,4,0.0,1


------------------
/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-02-09082023
--------------------------------------------------------------------------------
2


,petname,stage,step,timestamp,step_n,duration,experiment_id
0,01-true-shark,SetupTrench,STARTED,2023-08-09 06:49:04+00:00,0,21.0,2
1,01-true-shark,SetupTrench,FINISHED,2023-08-09 06:49:25+00:00,1,0.0,2
2,01-true-shark,ReadyTrench,STARTED,2023-08-09 06:49:25+00:00,2,0.0,2
3,01-true-shark,ReadyTrench,FINISHED,2023-08-09 06:49:25+00:00,3,12.0,2
4,01-true-shark,MillTrench,STARTED,2023-08-09 06:49:37+00:00,4,0.0,2


------------------
/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-15082023
--------------------------------------------------------------------------------
3


,petname,stage,step,timestamp,step_n,duration,experiment_id
0,01-fast-lark,SetupTrench,STARTED,2023-08-15 00:09:39+00:00,0,0.0,3
1,02-needed-toad,SetupTrench,STARTED,2023-08-15 00:09:39+00:00,1,1.0,3
2,03-boss-shrimp,SetupTrench,STARTED,2023-08-15 00:09:40+00:00,2,0.0,3
3,04-wired-colt,SetupTrench,STARTED,2023-08-15 00:09:40+00:00,3,0.0,3
4,05-grown-turtle,SetupTrench,STARTED,2023-08-15 00:09:40+00:00,4,0.0,3


------------------


In [48]:
SQL_CREATE_DETECTIONS_TABLE = """CREATE TABLE IF NOT EXISTS detections (
                                id INTEGER PRIMARY KEY,
                                petname VARCHAR(100) NOT NULL,
                                stage VARCHAR(100) NOT NULL,
                                step VARCHAR(100) NOT NULL,
                                feature VARCHAR(100) NOT NULL,
                                dpx_x INTEGER NOT NULL,
                                dpx_y INTEGER NOT NULL,
                                dm_x FLOAT NOT NULL,
                                dm_y FLOAT NOT NULL,
                                is_correct BOOL NOT NULL,
                                beam_type VARCHAR(100) NOT NULL,
                                fname VARCHAR(100) NOT NULL,
                                timestamp TIMESTAMP NOT NULL,
                                experiment_id INTEGER NOT NULL,
                                FOREIGN KEY (experiment_id) REFERENCES experiment (id)
                                
                            );"""

# create / connect to db
database = "fibsem.db"
conn = create_connection(database)

# create tables
create_table(conn, SQL_CREATE_DETECTIONS_TABLE)

In [49]:
%load_ext autoreload
%autoreload 2
from autolamella import config as cfg
import os

import pandas as pd
from autolamella.structures import Experiment
from autolamella.tools.data import calculate_statistics_dataframe

EXPERIMENTS = [
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-02-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-15082023",
]


df_exp = pd.read_sql("SELECT * FROM experiments", conn)
display(df_exp)


# 4, stage
# 5, detections
# 6, interactions

for PATH in EXPERIMENTS:

    dfs = calculate_statistics_dataframe(PATH)
    df = dfs[5]




    # # display(df)
    # # # format date as datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    print(EXP_ID)
    df["experiment_id"] = EXP_ID
    # # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])

    # # rename lamella to petname
    df = df.rename(columns={"lamella": "petname"})

    # # replace NA with 0
    df = df.fillna(0)
    
    display(df.head())

    df.to_sql('detections', conn, if_exists='append', index = False)

    print("------------------")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,id,name,project_id,date,user,sample_id,program,method,path
0,1,HANNAH-WAFFLE-01-09082023,1,2023-08-09 00:07:42.963713,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...
1,2,HANNAH-WAFFLE-02-09082023,1,2023-08-09 06:48:25.088992,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...
2,3,HANNAH-WAFFLE-01-15082023,1,2023-08-14 23:59:41.935888,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...


--------------------------------------------------------------------------------
1


,petname,stage,step,feature,dpx_x,dpx_y,dm_x,dm_y,is_correct,timestamp,beam_type,fname,experiment_id
0,01-busy-mite,MillUndercut,ALIGN_TRENCH,LamellaCentre,-208,222,-0.000054,5.781250e-05,False,2023-08-09 02:37:12+00:00,ELECTRON,ml-1691548599_7518098,1
1,01-busy-mite,MillUndercut,ALIGN_TRENCH,LamellaCentre,0,0,0.000000,0.000000e+00,True,2023-08-09 02:37:29+00:00,ION,ml-1691548636_7861617,1
2,01-busy-mite,MillUndercut,MILL_UNDERCUT_01,LamellaTopEdge,-72,2,-0.000007,1.953125e-07,False,2023-08-09 02:37:58+00:00,ION,ml-1691548664_5215123,1
3,01-busy-mite,MillUndercut,MILL_UNDERCUT_02,LamellaTopEdge,-78,245,-0.000008,2.392578e-05,False,2023-08-09 02:50:00+00:00,ION,ml-1691549385_885955,1
4,02-prime-guppy,MillUndercut,ALIGN_TRENCH,LamellaCentre,-225,252,-0.000059,6.562500e-05,False,2023-08-09 02:53:21+00:00,ELECTRON,ml-1691549588_1830385,1


------------------
--------------------------------------------------------------------------------
2


,petname,stage,step,feature,dpx_x,dpx_y,dm_x,dm_y,is_correct,timestamp,beam_type,fname,experiment_id
0,01-true-shark,MillUndercut,ALIGN_TRENCH,LamellaCentre,0,0,0.000000,0.000000,True,2023-08-09 07:00:08+00:00,ELECTRON,ml-1691564397_0947952,2
1,01-true-shark,MillUndercut,ALIGN_TRENCH,LamellaCentre,0,0,0.000000,0.000000,True,2023-08-09 07:00:24+00:00,ION,ml-1691564413_0397968,2
2,01-true-shark,MillUndercut,MILL_UNDERCUT_01,LamellaTopEdge,0,0,0.000000,0.000000,True,2023-08-09 07:00:51+00:00,ION,ml-1691564440_3788002,2
3,01-true-shark,MillUndercut,MILL_UNDERCUT_02,LamellaTopEdge,-82,253,-0.000008,0.000025,False,2023-08-09 07:10:16+00:00,ION,ml-1691565003_1505473,2


------------------
--------------------------------------------------------------------------------
3


,petname,stage,step,feature,dpx_x,dpx_y,dm_x,dm_y,is_correct,timestamp,beam_type,fname,experiment_id
0,01-fast-lark,MillUndercut,ALIGN_TRENCH,LamellaCentre,0,0,0.0,0.0,True,2023-08-15 03:04:50+00:00,ELECTRON,ml-1692068655_3507586,3
1,01-fast-lark,MillUndercut,ALIGN_TRENCH,LamellaCentre,0,0,0.0,0.0,True,2023-08-15 03:05:11+00:00,ION,ml-1692068695_1988711,3
2,01-fast-lark,MillUndercut,MILL_UNDERCUT_01,LamellaTopEdge,0,0,0.0,0.0,True,2023-08-15 03:05:40+00:00,ION,ml-1692068726_9891908,3
3,01-fast-lark,MillUndercut,MILL_UNDERCUT_02,LamellaTopEdge,0,0,0.0,0.0,True,2023-08-15 03:10:12+00:00,ION,ml-1692068999_1174848,3
4,01-fast-lark,MillUndercut,MILL_UNDERCUT_02,LamellaCentre,0,0,0.0,0.0,True,2023-08-15 03:14:46+00:00,ION,ml-1692069272_5425,3


------------------


In [54]:
SQL_CREATE_INTERACTIONS_TABLE = """CREATE TABLE IF NOT EXISTS interactions (
                                id INTEGER PRIMARY KEY,
                                petname VARCHAR(100) NOT NULL,
                                stage VARCHAR(100) NOT NULL,
                                step VARCHAR(100) NOT NULL,
                                type VARCHAR(100) NOT NULL,
                                subtype VARCHAR(100) NOT NULL, 
                                dm_x FLOAT NOT NULL,
                                dm_y FLOAT NOT NULL,
                                beam_type VARCHAR(100) NOT NULL,
                                timestamp TIMESTAMP NOT NULL,
                                experiment_id INTEGER NOT NULL,
                                FOREIGN KEY (experiment_id) REFERENCES experiment (id)
                                
                            );"""

# create / connect to db
database = "fibsem.db"
conn = create_connection(database)

# create tables
create_table(conn, SQL_CREATE_INTERACTIONS_TABLE)

In [65]:
%load_ext autoreload
%autoreload 2
from autolamella import config as cfg
import os

import pandas as pd
from autolamella.structures import Experiment
from autolamella.tools.data import calculate_statistics_dataframe

EXPERIMENTS = [
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-02-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-15082023",
]


df_exp = pd.read_sql("SELECT * FROM experiments", conn)
display(df_exp)


# 4, stage
# 5, detections
# 6, interactions

for PATH in EXPERIMENTS:

    dfs = calculate_statistics_dataframe(PATH)
    df = dfs[6]




    # # display(df)
    # # # format date as datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    # print(EXP_ID)
    df["experiment_id"] = EXP_ID
    # # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])

    # # rename lamella to petname
    df = df.rename(columns={"lamella": "petname"})

    # # replace NA with 0
    df = df.fillna(0)
    
    display(df[df.type=="MOVE"].head())

    df.to_sql('interactions', conn, if_exists='append', index = False)

    print("------------------")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,id,name,project_id,date,user,sample_id,program,method,path
0,1,HANNAH-WAFFLE-01-09082023,1,2023-08-09 00:07:42.963713,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...
1,2,HANNAH-WAFFLE-02-09082023,1,2023-08-09 06:48:25.088992,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...
2,3,HANNAH-WAFFLE-01-15082023,1,2023-08-14 23:59:41.935888,hannah,1,AutoLamella,waffle,C:/Users/Admin/Github/autolamella/autolamella/...


--------------------------------------------------------------------------------


,petname,stage,step,type,subtype,dm_x,dm_y,beam_type,timestamp,experiment_id
0,NULL,SystemSetup,SystemSetup,MOVE,STABLE,0.000153,-0.000232,ELECTRON,2023-08-09 00:09:13+00:00,1
1,NULL,SystemSetup,SystemSetup,MOVE,EUCENTRIC,0.000008,0.000308,ION,2023-08-09 00:09:37+00:00,1
2,NULL,SystemSetup,SystemSetup,MOVE,EUCENTRIC,0.000003,0.000258,ION,2023-08-09 00:09:51+00:00,1
10,06-united-urchin,MillTrench,MILL_TRENCH,MOVE,STABLE,-0.000041,-0.000056,ION,2023-08-09 01:50:37+00:00,1
11,06-united-urchin,MillTrench,MILL_TRENCH,MOVE,STABLE,-0.000027,-0.000018,ION,2023-08-09 01:51:04+00:00,1


------------------
--------------------------------------------------------------------------------


,petname,stage,step,type,subtype,dm_x,dm_y,beam_type,timestamp,experiment_id
0,NULL,SystemSetup,SystemSetup,MOVE,STABLE,0.000064,-0.000213,ION,2023-08-09 06:48:39+00:00,2


------------------
--------------------------------------------------------------------------------


,petname,stage,step,type,subtype,dm_x,dm_y,beam_type,timestamp,experiment_id
0,NULL,SystemSetup,SystemSetup,MOVE,EUCENTRIC,-1.300000e-07,0.000183,ION,2023-08-15 00:01:36+00:00,3
29,06-honest-eagle,ReadyTrench,FINISHED,MOVE,STABLE,-1.200000e-04,-0.000018,ION,2023-08-15 00:33:37+00:00,3
36,07-tough-collie,ReadyTrench,FINISHED,MOVE,STABLE,-5.240000e-05,-0.000066,ION,2023-08-15 00:34:37+00:00,3
55,01-fast-lark,ReadyLamella,SETUP_PATTERNS,MOVE,STABLE,-4.760000e-06,-0.000004,ION,2023-08-15 04:42:47+00:00,3
56,01-fast-lark,ReadyLamella,SETUP_PATTERNS,MOVE,EUCENTRIC,-4.870000e-06,-0.000007,ION,2023-08-15 05:00:20+00:00,3


------------------
